In [1]:
import requests
import csv
import time

BASE_URL = "https://vinfastauto.com/vn_vi/get-locator"
OUTPUT_CSV = "vinfast_connector_count.csv"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}
TIMEOUT = 7
SLEEP = 0.05  # tránh rate limit

def _count_connectors(obj):
    """Đếm tổng số phần tử trong mọi mảng key='connectors' (đệ quy toàn bộ cây)."""
    if obj is None:
        return 0
    total = 0
    if isinstance(obj, dict):
        for k, v in obj.items():
            if isinstance(k, str) and k.lower() == "connectors" and isinstance(v, list):
                # Đếm số phần tử thực sự là 'cổng'
                # (ở cấu trúc bạn đưa, mỗi phần tử connectors là 1 dict có 'id'/'standard'...)
                total += sum(1 for x in v if x is not None)
            total += _count_connectors(v)
    elif isinstance(obj, list):
        for it in obj:
            total += _count_connectors(it)
    return total

def fetch_connector_count(station_id: int):
    url = f"{BASE_URL}/{station_id}"
    try:
        r = requests.get(url, timeout=TIMEOUT, headers=HEADERS)
        if r.status_code != 200:
            return None
        root = r.json() or {}
        payload = root.get("data", {}) or {}
        nested = payload.get("data", {}) or {}

        # Ưu tiên đếm ở tầng sâu nhất (tránh đếm trùng nếu bên ngoài cũng có tóm tắt)
        cnt = _count_connectors(nested)
        if cnt == 0:
            cnt = _count_connectors(payload)
        if cnt == 0:
            cnt = _count_connectors(root)

        # Nếu vẫn 0, coi như trạm/ID không hợp lệ hoặc không có cổng
        return {"id": station_id, "connector_count": cnt}
    except requests.RequestException:
        return None
    except ValueError:
        return None

def main():
    results = []
    # Điều chỉnh dải ID theo nhu cầu của bạn
    for sid in range(14000, 30001):
        info = fetch_connector_count(sid)
        if info and info["connector_count"] > 0:
            results.append(info)
        time.sleep(SLEEP)

    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["id", "connector_count"])
        writer.writeheader()
        writer.writerows(results)

    print(f"Đã lưu {len(results)} dòng vào {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


Đã lưu 8133 dòng vào vinfast_connector_count.csv
